In [5]:
import numpy as np
from collections import Counter

def ellipsoidal_quantile(data, q):
    cov = np.cov(data.T)
    inv_cov = np.linalg.inv(cov)
    mean = np.mean(data, axis=0)
    
    _, eig_vects = np.linalg.eig(cov)

    # distances from the mean in the 'circular' space
    circled_data = sorted(np.sqrt([np.dot(np.dot((row-mean).T, inv_cov), (row-mean)) for row in data]))
    n = len(circled_data)
    n_at_distance = Counter(circled_data)

    # calculate qlevel
    qlevel = [0]
    for distance in reversed(sorted(n_at_distance.keys())):
        qlevel.append(qlevel[-1]+n_at_distance[distance])
    qlevel = [(n-float(i))/n for i in qlevel]
    qlevel = qlevel[1:-1]
    qlevel = np.array(qlevel)

    # ========================
    
    index_outer = qlevel[qlevel>q].shape[0]+1
    index_inner = index_outer+1
    
    qlevel = [1] + list(qlevel)
    distances = list(reversed(sorted(set(circled_data))))
    qo = qlevel[index_outer-1]
    do = distances[index_outer-1]

    if index_outer == len(distances):
        qi, di = 0, 0
    else:
        qi = qlevel[index_inner-1]
        di = distances[index_inner-1]

    quantile_distance = di + (do-di)*((q-qi)/(qo-qi))
    quantile_distance = quantile_distance**2

    new_covariance =  quantile_distance * cov

    # ========================

    vals, vects = np.linalg.eig(new_covariance)
    radii = np.sqrt(vals)

    # ========================
    
    print mean
    print radii
    print eig_vects
    print '========'


In [6]:
data = np.array([(1, 3), (4, 4), (5, 6), (5, 15), (20, 10)])
ellipsoidal_quantile(data, 0.90)

[ 7.   7.6]
[ 13.8019558    7.60939241]
[[ 0.9307145  -0.36574653]
 [ 0.36574653  0.9307145 ]]


In [7]:
data = np.array([(1, 1), (0, 0), (2, 2)])
ellipsoidal_quantile(data, 0.9)

LinAlgError: Singular matrix

In [8]:
data = np.array([(0, -1), (-1, 0), (0, 1), (1, 0)])
ellipsoidal_quantile(data, 0.9)
ellipsoidal_quantile(data, 0.8)
ellipsoidal_quantile(data, 0.75)

[ 0.  0.]
[ 0.9  0.9]
[[ 1.  0.]
 [ 0.  1.]]
[ 0.  0.]
[ 0.8  0.8]
[[ 1.  0.]
 [ 0.  1.]]
[ 0.  0.]
[ 0.75  0.75]
[[ 1.  0.]
 [ 0.  1.]]


In [9]:
data = np.array([(0, -1), (1, 3), (-4, 0), (5, 1), (6, 7), (11, 2), (3, 3), (1, 7)])
ellipsoidal_quantile(data, 0.9)
ellipsoidal_quantile(data, 0.8)

[ 2.875  2.75 ]
[ 7.92276089  4.78140648]
[[ 0.96289942 -0.26986054]
 [ 0.26986054  0.96289942]]
[ 2.875  2.75 ]
[ 7.54609152  4.55408555]
[[ 0.96289942 -0.26986054]
 [ 0.26986054  0.96289942]]


In [10]:
data = np.array([(1, 1, 1), (2, 3, 4), (6, 7, 8), (1, 0, 0), (-2, 3, 4), (9, -10, 1)])
ellipsoidal_quantile(data, 0.8)

[ 2.83333333  0.66666667  3.        ]
[ 12.59780615   7.36977253   2.15709836]
[[-0.3746164   0.81857494 -0.43542809]
 [ 0.87856714  0.16332574 -0.44882567]
 [ 0.29628083  0.55069026  0.78035755]]
